# TD de Machine Learning : `Prédiction de la Note moyenne des livres`
### Exercice 1 :
Les données d'entrées sont sauvegardées dans l'index `books` sur Elasticsearch.
Créer une base de données d'entraînement et de test.
Vous enregistrerez la base d'entraînement dans un index d'elasticsearch, qui portera le nom suivant :

- `uppa_2025_train_nom_prenom`


### Exercice 2 :    
Créer un modèle `more_like_this` pour la prédiction de la note moyenne du livre (`average_rating`)

### Exercice 3 :     
Créer un modèle `more_like_this` par langue pour la prédiction de la note moyenne du livre (`average_rating`). Vous devrez dans un premier temps identifier la langue de chaque livre.

## Installation des librairies

In [ ]:
!pip install elasticsearch

In [ ]:
!pip install elasticsearch-dsl

In [ ]:
!pip install wget

In [ ]:
!pip install langdetect

## Importation des librairies

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch_dsl import Search
import pandas as pd
import numpy as np
from langdetect import detect
import wget
from google.colab import drive
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

# Exercice 1


In [ ]:
# @title **Connexion à Elasticsearch**

es = Elasticsearch(
    hosts=['http://formakuntza-kibana.hupi.io:9200/'],
    basic_auth=('student', 'gCzOD2pVnCqVpd13')
)

try:
    # Récupérer les informations du cluster
    info = es.info()

    # Afficher des informations intéressantes
    print(f"Connexion réussie au cluster : {info['cluster_name']}")
    print(f"Version d'Elasticsearch : {info['version']['number']}")

    # Afficher les indices existants
    indices = es.cat.indices(format="json")
    print(f"Nombre d'index dans le cluster : {len(indices)}")

except Exception as e:
    print(f"Erreur de connexion : {e}")

Connexion réussie au cluster : docker-cluster
Version d'Elasticsearch : 7.17.24
Nombre d'index dans le cluster : 11


In [ ]:
# @title **Lecture de l'index "books" comme les données d'entrées**

# @title **Afficher l'index "ecommerce"**
index_name = "books"

s = Search(using=es, index=index_name).query("match_all")

# Utilisation de la méthode scan pour récupérer tous les résultats rapidement
results = s.scan()

# Convertir les résultats en DataFrame
data = [hit.to_dict() for hit in results]
df = pd.json_normalize(data)

# Explorer les données
print(f"Show dataframe shape: {df.shape}")
print(f"Show dataframe first 10 rows: \n {df.head()}")

Show dataframe shape: (6810, 12)
Show dataframe first 10 rows: 
                                            thumbnail  published_year  num_pages                                        description  average_rating  \
0  http://books.google.com/books/content?id=KQZCP...          2004.0      247.0  A NOVEL THAT READERS and critics have been eag...            3.85   
1  http://books.google.com/books/content?id=gA5GP...          2000.0      241.0  A new 'Christie for Christmas' -- a full-lengt...            3.83   
2  http://books.google.com/books/content?id=OmQaw...          1982.0      479.0  Volume Two of Stephen Donaldson's acclaimed se...            3.97   
3  http://books.google.com/books/content?id=FKo2T...          1993.0      512.0  A memorable, mesmerizing heroine Jennifer -- b...            3.93   
4  http://books.google.com/books/content?id=XhQ5X...          2002.0      170.0  Lewis' work on the nature of love divides love...            4.15   

            title         isbn13  

In [ ]:
# @title **Diviser les données en ensembles d'entraînement (80%) et de test (20%)**
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Taille de l'ensemble d'entraînement (train_df) : {train_df.shape}")
print(f"Taille de l'ensemble de test (test_df) : {test_df.shape}")

Taille de l'ensemble d'entraînement (train_df) : (5448, 12)
Taille de l'ensemble de test (test_df) : (1362, 12)


In [ ]:
# @title **Créer l'index pour les données d'entraînement dans Elasticsearch**
index_name = 'uppa_2025_train_dassance_kattin'

# Mapping for the index (modify according to your data structure)
mapping = {
    "mappings": {
        "properties": {
            "isbn13": {"type": "long"},
            "title": {"type": "text"},
            "authors": {"type": "text"},
            "categories": {"type": "keyword"},
            "description": {"type": "text"},
            "published_year": {"type": "long"},
            "average_rating": {"type": "double"},
            "num_pages": {"type": "long"},
            "ratings_count": {"type": "long"}
        }
    }
}

# Create the index with the specified mapping
response = (es.options(ignore_status=400)
              .indices.create(index=index_name, body=mapping))
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'uppa_2025_dassance_kattin'}


In [ ]:
# @title **Ecriture des données d'entraînement dans Elasticsearch**

def df_to_es(df):
    for record in df.to_dict(orient='records'):
        yield {
            "_index": index_name,
            "_source": record
        }

# Bulk insert the data into Elasticsearch
col_train = ["isbn13", "title", "authors", "categories", "description", "published_year", "average_rating", "num_pages", "ratings_count"]
df_to_index = train_df[col_train].dropna()
success, failed = bulk(es, df_to_es(df_to_index), raise_on_error=False)  # Set raise_on_error to False

# Print the result
print(f"Successfully indexed {success} documents.")
print(f"Failed to index {failed} documents.")

# Print details of failed documents
for item in failed:
    print(f"Failed document: {item}")

Successfully indexed 5119 documents.
Failed to index [] documents.


# Exercice 2 :
Prédiction de la note moyenne des livres.

### Question 1 :   
Calculer un score pour un livre que vous auriez choisi.
L'objectif est de prendre en main la requête `more_like_this`

### Question 2 :     
Créer un modèle sur la base d'entraînement de `train_df` que vous testerez pour l'ensemble des livres présents dans `test_df`. Vous mettrez également en place des métriques afin d'évaluer et améliorer vos différents modèles.

### Question 3 :     
Créer un modèle par langue sur la base d'entraînement de `train_df` que vous testerez pour l'ensemble des livres présents dans `test_df`. Vous mettrez également en place des métriques afin d'évaluer et améliorer vos différents modèles.

## Question 1 : Détection des similarités avec `more_like_this`
Documentation : https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html

In [ ]:
###############################################   Select an item for evaluation

response = es.search(index="uppa_2025_train_dassance_kattin", body={
    "query": {
        "match": {
            "isbn13": 9780002005883
        }
    },
    "explain": "true"
})

print(f"Response: {response} \n")
hits = response['hits']['hits']
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(f"Dataframe view: \n{df}")

Response: {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_shard': '[uppa_2025_dassance_kattin][0]', '_node': 'w4J3bTfdQgWTrMMKXDMARw', '_index': 'uppa_2025_dassance_kattin', '_type': '_doc', '_id': '7dCwT5QB5IuiTNqkUQzC', '_score': 1.0, '_source': {'isbn13': 9780002005883, 'title': 'Gilead', 'authors': 'Marilynne Robinson', 'categories': 'Fiction', 'description': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between

In [ ]:
df["description"].iloc[0] # to copy the content of the description

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [ ]:
###############################################   Apply the more_like_this function

response = es.search(index="uppa_2025_train_dassance_kattin", body={
    "query": {
        "more_like_this": {
            "fields": ["title", "author", "description"],
            "like": "A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world has to offer. At its heart is a tale of the sacred bonds between fathers and sons, pitch-perfect in style and story, set to dazzle critics and readers alike."
        }
    },
    "explain": "true"
})

print(f"Response: {response} \n")
hits = response['hits']['hits']
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(f"Dataframe view: \n{df}")

Response: {'took': 219, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3405, 'relation': 'eq'}, 'max_score': 44.67813, 'hits': [{'_shard': '[uppa_2025_dassance_kattin][0]', '_node': 'w4J3bTfdQgWTrMMKXDMARw', '_index': 'uppa_2025_dassance_kattin', '_type': '_doc', '_id': '7dCwT5QB5IuiTNqkUQzC', '_score': 44.67813, '_source': {'isbn13': 9780002005883, 'title': 'Gilead', 'authors': 'Marilynne Robinson', 'categories': 'Fiction', 'description': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of t

In [ ]:
# explanation of similarities
response["hits"]["hits"]

[{'_shard': '[uppa_2025_dassance_kattin][0]',
  '_node': 'w4J3bTfdQgWTrMMKXDMARw',
  '_index': 'uppa_2025_dassance_kattin',
  '_type': '_doc',
  '_id': '7dCwT5QB5IuiTNqkUQzC',
  '_score': 44.67813,
  '_source': {'isbn13': 9780002005883,
   'title': 'Gilead',
   'authors': 'Marilynne Robinson',
   'categories': 'Fiction',
   'description': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by

Le livre le plus similaire selon le contenu, l'auteur et le titre est le livre "John Adams" de David McCullough dans la catégorie "Biography & Autobiography", qui est noté à 4.06. Nous aurions donc fait une erreur de : 4.06 - 3.85 = 0.21


In [ ]:
# @title Correction : pour aller plus loin une autre façon d'écrire la requête.

# Utiliser more_like_this pour construire un modèle de similarité
def more_like_this_model(book_id):
    response = es.search(index="books", body={
        "query": {
            "more_like_this": {
                "fields": ["title", "author", "description"],
                "like": [
                    {
                        "_index": "books",
                        "_id": book_id
                    }
                ],

                "min_term_freq": 1,
                "max_query_terms": 12
            }
        }
    })
    # Récupérer les informations relatives aux livres similaires
    similar_books_info = [hit['_source'] for hit in response['hits']['hits']]
    # [hit['_id'] for hit in response['hits']['hits']]
    return similar_books_info

# Tester la fonction more_like_this_model_with_info
book_id = "4896RpQB5IuiTNqk9FCO"  # Remplacez par l'ID d'un livre spécifique
similar_books_info = more_like_this_model(book_id)
print(f"Books similar to book {book_id}: {similar_books_info} \n")

# Afficher les informations sous forme de DataFrame
df_similar_books = pd.DataFrame(similar_books_info)
print(df_similar_books)

Books similar to book 4896RpQB5IuiTNqk9FCO: [{'thumbnail': 'http://books.google.com/books/content?id=GXWJDQAAQBAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api', 'published_year': 1996, 'num_pages': 608, 'description': "Winner of the Pulitzer Prize, the Howells Medal, and the National Book Critics Circle Award In John Updike's fourth and final novel about Harry “Rabbit” Angstrom, the hero has acquired a Florida condo, a second grandchild, and a troubled, overworked heart. His son, Nelson, is behaving erratically; his daughter-in-law, Pru, is sending him mixed signals; and his wife, Janice, decides in midlife to return to the world of work. As, through the year of 1989, Reagan's debt-ridden, AIDS-plagued America yields to that of the first George Bush, Rabbit explores the bleak terrain of late middle age, looking for reasons to live and opportunities to make peace with a remorselessly accumulating past.", 'average_rating': 3.97, 'title': 'Rabbit at Rest', 'isbn13': 9780449911945, 'isb

### Correction : pour aller plus loin
Recherche des termes avec un second modèle `multi_match`
Documentation : https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html

In [ ]:
###############################################    Apply the multi_match function

response = es.search(index="uppa_2025_train_dassance_kattin", body={
    "query": {
        "multi_match": {
            "query": "A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world has to offer. At its heart is a tale of the sacred bonds between fathers and sons, pitch-perfect in style and story, set to dazzle critics and readers alike.",
            "fields": ["description"]
        }
    }
})

print(response)
hits = response['hits']['hits']
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

{'took': 85, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5072, 'relation': 'eq'}, 'max_score': 479.51166, 'hits': [{'_index': 'uppa_2025_dassance_kattin', '_type': '_doc', '_id': '7dCwT5QB5IuiTNqkUQzC', '_score': 479.51166, '_source': {'isbn13': 9780002005883, 'title': 'Gilead', 'authors': 'Marilynne Robinson', 'categories': 'Fiction', 'description': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought

## Question 2

In [ ]:
def get_similar_document(row):
    # Extraire les informations pertinentes de la ligne du dataframe
    title = row['title']
    author = row['authors']
    description = row['description']

    # Requête more_like_this pour Elasticsearch
    response = es.search(index="uppa_2025_train_dassance_kattin", body={
        "query": {
            "more_like_this": {
                "fields": ["title", "authors", "description"],
                "like": f"{title} {author} {description}"
            }
        },
        "explain": "true"
    })

    # Récupérer les documents similaires
    hits = response['hits']['hits']

    if hits:
        # Extraire les informations du premier résultat le plus pertinent
        most_similar_doc = hits[0]['_source']
        # Renvoyer la valeur de la colonne 'average_rating'
        return most_similar_doc.get('average_rating', None)
    else:
        return None

In [ ]:
test_df['similar_average_rating'] = test_df.apply(get_similar_document, axis=1)

In [ ]:
# compute metrics
mae = np.mean(np.abs(test_df['similar_average_rating'] - test_df['average_rating']))
print(f"Erreur absolue moyenne (MAE) : {mae}")

mse = np.mean((test_df['similar_average_rating'] - test_df['average_rating']) ** 2)
print(f"Erreur quadratique moyenne (MSE) : {mse}")

rmse = np.sqrt(mse)
print(f"Racine de l'erreur quadratique moyenne (RMSE) : {rmse}")

Erreur absolue moyenne (MAE) : 0.28680250783699063
Erreur quadratique moyenne (MSE) : 0.17300156739811912
Racine de l'erreur quadratique moyenne (RMSE) : 0.4159345710542935


## Question 3

### Détection de la langue

In [ ]:
# Fonction pour détecter la langue d'un titre de livre
def detect_language(description):
    try:
        return detect(description)
    except:
        return 'unknown'

In [ ]:
# Ajouter une colonne 'language' au DataFrame
train_df['language'] = train_df['description'].apply(detect_language)
test_df['language'] = test_df['description'].apply(detect_language)

In [ ]:
train_df.head(2)

,thumbnail,published_year,num_pages,description,average_rating,title,isbn13,isbn10,categories,ratings_count,authors,subtitle,language
4235,http://books.google.com/books/content?id=W3dGP...,1992.0,288.0,NaN,3.93,The Egg and I,9780704102477,704102471,Farm life,6223.0,Betty MacDonald,NaN,unknown
6602,http://books.google.com/books/content?id=8ClIP...,2000.0,288.0,"In the year 2100, mankind on Earth, settlers i...",4.09,The Gods Themselves,9781857989342,1857989341,Science fiction,41304.0,Isaac Asimov,NaN,en


In [ ]:
# Vérifier la distribution des langues
print(train_df['language'].value_counts())

language
en         5199
unknown     220
de           10
id            8
nl            3
da            2
vi            1
af            1
tl            1
sl            1
ja            1
ro            1
Name: count, dtype: int64


In [ ]:
for index, row in train_df.iterrows():
    id_book = row['isbn13']  # ID du document dans Elasticsearch
    language = row['language']  # Langue à mettre à jour

    # Utiliser update_by_query pour mettre à jour le champ 'language'
    update_query = {
        "script": {
            "source": "ctx._source.language = params.language",
            "params": {
                "language": language
            }
        },
        "query": {
            "term": {
                "isbn13": id_book  # Filtrer sur id_book
            }
        }
    }

    # Exécuter la requête
    response = es.update_by_query(index="uppa_2025_train_dassance_kattin", body=update_query)

    print(f"Updated language for id_book {id_book}: {language}. Response: {response}")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Updated language for id_book 9780099481249: en. Response: {'took': 6, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Updated language for id_book 9780192833594: en. Response: {'took': 14, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Updated language for id_book 9780446600347: en. Response: {'took': 6, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis'

### Evaluation de la similarité

In [ ]:
def get_similar_document_by_language(row):
    # Extraire les informations pertinentes de la ligne du dataframe
    title = row['title']
    authors = row['authors']
    description = row['description']
    language = row['language']

    # Requête more_like_this pour Elasticsearch, en incluant un filtre sur la langue
    response = es.search(index="uppa_2025_train_dassance_kattin", body={
        "query": {
            "bool": {
                "must": {
                    "more_like_this": {
                        "fields": ["title", "authors", "description"],
                        "like": f"{title} {authors} {description}"
                    }
                },
                "filter": {
                    "term": {
                        "language": language  # Filtrer par langue
                    }
                }
            }
        },
        "explain": "true"
    })

    # Récupérer les documents similaires
    hits = response['hits']['hits']

    if hits:
        # Extraire les informations du premier résultat le plus pertinent
        most_similar_doc = hits[0]['_source']
        # Renvoyer la valeur de la colonne 'average_rating'
        return most_similar_doc.get('average_rating', None)
    else:
        return None

In [ ]:
test_df['lang_similar_average_rating'] = test_df.apply(get_similar_document_by_language, axis=1)

# Afficher le dataframe avec la nouvelle colonne 'lang_similar_average_rating'
print(test_df.head())

                                              thumbnail  published_year  num_pages                                        description  \
1632  http://books.google.com/books/content?id=dTHrm...          2004.0      528.0  According to the DC cops, the fiery destructio...   
3891  http://books.google.com/books/content?id=ozQ8R...          1958.0      260.0  A story of a Chinese peasant and his passionat...   
2115  http://books.google.com/books/content?id=E-GwA...          1993.0      185.0  In a world marked by full-scale neighborhood w...   
2404  http://books.google.com/books/content?id=LUrJI...          2001.0      256.0  This two-part profile of the late Nobel Prize-...   
3260  http://books.google.com/books/content?id=HGrNS...          1994.0      720.0  Fans of Jurassic Park and Rising Sun, the book...   

      average_rating                                        title         isbn13     isbn10                 categories  ratings_count  \
1632            3.76                    

In [ ]:
# compute metrics
mae = np.mean(np.abs(test_df['lang_similar_average_rating'] - test_df['average_rating']))
print(f"Erreur absolue moyenne (MAE) : {mae}")

mse = np.mean((test_df['lang_similar_average_rating'] - test_df['average_rating']) ** 2)
print(f"Erreur quadratique moyenne (MSE) : {mse}")

rmse = np.sqrt(mse)
print(f"Racine de l'erreur quadratique moyenne (RMSE) : {rmse}")

Erreur absolue moyenne (MAE) : 0.28802683504340965
Erreur quadratique moyenne (MSE) : 0.17682186266771902
Racine de l'erreur quadratique moyenne (RMSE) : 0.4205019175553413
